
EJERCICIO 4



In [1]:
# Instalamos las librerías necesarias
!pip install beautifulsoup4
!pip install requests
!pip install tensorflow
!pip install keras
!pip install nltk
!pip install kaggle

# Importamos las librerías necesarias
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import pad_sequences, to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:

# Descomprimir el dataset
!unzip /content/wiki_movie_plots_deduped.csv.zip

# Cargar el dataset
data = pd.read_csv('/content/wiki_movie_plots_deduped.csv.zip')

# Mostrar las primeras filas del dataset
print(data.head())



Archive:  /content/wiki_movie_plots_deduped.csv.zip
  inflating: wiki_movie_plots_deduped.csv  
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...   
1  https://en.wikipedia.org/wiki/Love_by

In [8]:
# Preprocesar los datos
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Aplicar preprocesamiento a la columna de texto del dataset
data['processed_text'] = data['Plot'].apply(preprocess_text)

# Concatenar todas las oraciones en un solo corpus
corpus = [word for sentence in data['processed_text'] for word in sentence]

# Definir la longitud de la secuencia de entrenamiento
train_len = 4

# Crear las secuencias de texto
text_sequences = []
for i in range(train_len, len(corpus)):
    seq = corpus[i-train_len:i]
    text_sequences.append(seq)



In [ ]:
# Preprocesar los datos
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens



In [ ]:
# Aplicar preprocesamiento a la columna de texto del dataset
data['processed_text'] = data['Plot'].apply(preprocess_text)

# Concatenar todas las oraciones en un solo corpus
corpus = [word for sentence in data['processed_text'] for word in sentence]

# Definir la longitud de la secuencia de entrenamiento
train_len = 4



In [ ]:
# Crear las secuencias de texto
text_sequences = []
for i in range(train_len, len(corpus)):
    seq = corpus[i-train_len:i]
    text_sequences.append(seq)

# Tokenizar las secuencias de texto
tok = Tokenizer()
tok.fit_on_texts(text_sequences)



In [ ]:
# Convertir las palabras a números
sequences = tok.texts_to_sequences(text_sequences)

# Convertir a numpy array
arr_sequences = np.array(sequences)

# Separar input y target
x_data = arr_sequences[:,:-1]
y_data_int = arr_sequences[:,-1]

# Obtener el tamaño del vocabulario
vocab_size = len(tok.word_counts) + 1


In [ ]:

# Transformar los datos a one hot encoding
y_data = to_categorical(y_data_int, num_classes=vocab_size)

# Crear el modelo
input_seq_len = x_data.shape[1]
output_size = vocab_size

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=input_seq_len))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


In [ ]:

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
hist = model.fit(x_data, y_data, epochs=10, validation_split=0.2)


In [ ]:

# Graficar los resultados
epoch_count = range(1, len(hist.history['accuracy']) + 1)
sns.lineplot(x=epoch_count, y=hist.history['accuracy'], label='train')
sns.lineplot(x=epoch_count, y=hist.history['val_accuracy'], label='valid')
plt.show()

# Función para predecir la próxima palabra
def model_response(human_text):
    encoded = tok.texts_to_sequences([human_text])[0]
    encoded = pad_sequences([encoded], maxlen=input_seq_len, padding='pre')
    y_hat = model.predict(encoded).argmax(axis=-1)
    out_word = ''
    for word, index in tok.word_index.items():
        if index == y_hat:
            out_word = word
            break
    return human_text + ' ' + out_word


In [ ]:

# Prueba de la función de predicción
print(model_response("The story of a young"))

# Generar secuencias nuevas
def generate_seq(model, tokenizer, seed_text, max_length, n_words):
    output_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([output_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        y_hat = model.predict(encoded).argmax(axis=-1)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_hat:
                out_word = word
                break
        output_text += ' ' + out_word
    return output_text


In [ ]:

# Prueba de la función de generación de secuencias
input_text = "A story about"
print(generate_seq(model, tok, input_text, max_length=3, n_words=5))

# Conclusiones
print("\nConclusiones:")
print("El modelo entrenado puede predecir la próxima palabra en una secuencia dada. Sin embargo, el rendimiento del modelo podría mejorarse con más datos y más épocas de entrenamiento. Además, se pueden explorar arquitecturas más complejas para mejorar la precisión de las predicciones.")
